In [1]:
import unittest

In [2]:
class SetUpAndTearDown(unittest.TestCase):
    def setUp(self):
        print('Setup')

    def tearDown(self):
        print('TearDown')

    def test_pass(self):
        print('Passing test')

    def test_error(self):
        print('Test resulting in an error')
        raise ValueError()

    def test_fail(self):
        print('Failing test')
        self.fail()

In [3]:
unittest.main(argv=['python', 'SetUpAndTearDown'], exit=False)

EF.

Setup
Test resulting in an error
TearDown
Setup
Failing test
TearDown
Setup
Passing test
TearDown



ERROR: test_error (__main__.SetUpAndTearDown)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-2-9f86f77f8664>", line 13, in test_error
    raise ValueError()
ValueError

FAIL: test_fail (__main__.SetUpAndTearDown)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-2-9f86f77f8664>", line 17, in test_fail
    self.fail()
AssertionError: None

----------------------------------------------------------------------
Ran 3 tests in 0.008s

FAILED (failures=1, errors=1)


In [5]:
class InvalidSetUp(unittest.TestCase):
    def setUp(self):
        self.stream = open('README.md', 'r')
        print('SetUp')
        raise Exception()

    def tearDown(self):
        self.stream.close()
        print('TearDown')

    def test_one(self):
        print('Test')

In [6]:
unittest.main(argv=['python', 'InvalidSetUp'], exit=False)

E

SetUp



ERROR: test_one (__main__.InvalidSetUp)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-5-7712b10ca369>", line 5, in setUp
    raise Exception()
Exception

----------------------------------------------------------------------
Ran 1 test in 0.004s

FAILED (errors=1)


In [8]:
class BetterSetUp(unittest.TestCase):
    def setUp(self):
        self.stream = open('README.md', 'r')
        self.addCleanup(lambda: print('Stream closed'))
        self.addCleanup(self.stream.close)
        self.addCleanup(lambda: print('Closing stream'))
        print('SetUp')
        raise Exception()

    def test_one(self):
        print('Test')

In [9]:
unittest.main(argv=['python', 'BetterSetUp'], exit=False)

E

SetUp
Closing stream
Stream closed



ERROR: test_one (__main__.BetterSetUp)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-8-f6f98e31bdaa>", line 8, in setUp
    raise Exception()
Exception

----------------------------------------------------------------------
Ran 1 test in 0.007s

FAILED (errors=1)


In [14]:
class SetUpClassAndTearDownClass(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        print('SetupClass')

    @classmethod
    def tearDownClass(cls):
        print('TearDownClass')
        
    def setUp(self):
        print('Setup')

    def tearDown(self):
        print('TearDown')

    def test_pass(self):
        print('Passing test')

    def test_error(self):
        print('Test resulting in an error')
        raise ValueError()

In [15]:
unittest.main(argv=['python', 'SetUpClassAndTearDownClass'], exit=False)

E.

SetupClass
Setup
Test resulting in an error
TearDown
Setup
Passing test
TearDown
TearDownClass



ERROR: test_error (__main__.SetUpClassAndTearDownClass)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-14-ba24c2099e25>", line 21, in test_error
    raise ValueError()
ValueError

----------------------------------------------------------------------
Ran 2 tests in 0.008s

FAILED (errors=1)


In [20]:
import sys

class Skipping(unittest.TestCase):
    @unittest.skip('demonstrating skipping')
    def test_noting(self):
        self.fail('should not be executed')

    @unittest.skipUnless(sys.platform.startswith('win'),
                         'requires Windows')
    def test_windows_support(self):
        pass

    @unittest.skipIf(sys.version.startswith('3'),
                     'requires Python 2')
    def test_python_2_support(self):
        pass

    @unittest.expectedFailure
    def test_expected_failure(self):
        self.fail('should be executed')

In [21]:
unittest.main(argv=['python', 'Skipping'], exit=False)

xsss
----------------------------------------------------------------------
Ran 4 tests in 0.009s

OK (skipped=3, expected failures=1)


In [26]:
class SubTests(unittest.TestCase):
    def test_subtest(self):
        for a, b, expected in [
            (0, 0, 0),
            (2, 2, 4),
            (2, 5, 5),
            (2, 7, 7),
        ]:
            with self.subTest(a=a, b=b, expected=expected):
                print('Subtest')
                got = a + b
                self.assertEqual(got, expected)

    def setUp(self):
        print('SetUp')

    def tearDown(self):
        print('TearDown')

In [27]:
unittest.main(argv=['python', 'SubTests'], exit=False)

SetUp
Subtest
Subtest
Subtest
Subtest
TearDown



FAIL: test_subtest (__main__.SubTests) (a=2, b=5, expected=5)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-26-090ef11344cb>", line 12, in test_subtest
    self.assertEqual(got, expected)
AssertionError: 7 != 5

FAIL: test_subtest (__main__.SubTests) (a=2, b=7, expected=7)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-26-090ef11344cb>", line 12, in test_subtest
    self.assertEqual(got, expected)
AssertionError: 9 != 7

----------------------------------------------------------------------
Ran 1 test in 0.002s

FAILED (failures=2)


# Mocking

In [28]:
%%writefile my_module.py
import os


DEFAULT_EXTENSION = '.txt'


def my_remove(filename):
    if '.' not in filename:
        filename += DEFAULT_EXTENSION
    os.remove(filename)


Writing my_module.py


In [37]:
%%writefile test_my_module.py
import unittest
from unittest import mock
from my_module import my_remove

class PatchingTestCase(unittest.TestCase):
    @mock.patch('my_module.os')
    def test_provided_extension_should_be_used(self, os_mock):
        my_remove('file.md')
        os_mock.remove.assert_called_once_with('file.md')

    @mock.patch('my_module.os')
    def test_when_extension_is_missing_then_use_default_one(self, os_mock):
        my_remove('file')
        os_mock.remove.assert_called_once_with('file.txt')
        
    @mock.patch('my_module.os')
    def test_hidden_files(self, os_mock):
        my_remove('.file')
        os_mock.remove.assert_called_once_with('.file.txt')
        
if __name__ == '__main__':
    unittest.main()

Overwriting test_my_module.py


In [38]:
!python3 test_my_module.py

F..
FAIL: test_hidden_files (__main__.PatchingTestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/usr/lib/python3.5/unittest/mock.py", line 1159, in patched
    return func(*args, **keywargs)
  File "test_my_module.py", line 19, in test_hidden_files
    os_mock.remove.assert_called_once_with('.file.txt')
  File "/usr/lib/python3.5/unittest/mock.py", line 805, in assert_called_once_with
    return self.assert_called_with(*args, **kwargs)
  File "/usr/lib/python3.5/unittest/mock.py", line 794, in assert_called_with
    raise AssertionError(_error_message()) from cause
AssertionError: Expected call: remove('.file.txt')
Actual call: remove('.file')

----------------------------------------------------------------------
Ran 3 tests in 0.006s

FAILED (failures=1)
